In [0]:
import os
path = "/content/drive/My Drive/NLP/sentiment_compete"
os.chdir(path)

In [2]:
!pip install transformers

     |████████████████████████████████| 573kB 4.6MB/s 
     |████████████████████████████████| 1.0MB 19.5MB/s 
     |████████████████████████████████| 3.7MB 29.3MB/s 
     |████████████████████████████████| 870kB 56.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.40-cp36-none-any.whl size=893351 sha256=3c942dd8c1ce56ca92fdee335f7fb1ecbdb654e2c2fe3415fd247ee904eb9e36
  Stored in directory: /root/.cache/pip/wheels/ba/44/6e/939fb8625572dca96344566c5167bc4f8ab079294509898141
Successfully built sacremoses


In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from processData import DataProcessor
from transformers import BertTokenizer,BertConfig
from transformers import BertModel
from config import Config
import torch
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from executor import ModelExcuter
from model.bert_model_base import BertModel_Base
from model.robert_model_base import RoBertModel_Base
from model.ernie_model_base import ErnieModel_Base
from model.ernie_model_pool_last3 import Ernie_PoolLast3_Model
from model.ernie_poollast3_multidp import Ernie_poollast3_multidp

SEED = 12345

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [15]:
config = Config()
bert_config = BertConfig.from_pretrained(config.bert_config_path, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(config.bert_vocab_path)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [0]:
dataProcessor = DataProcessor(tokenizer, config.max_seq_len, config.device)

In [8]:
train_dataset = dataProcessor.get_train_feature(config.train_data_path)
dev_dataset = dataProcessor.get_dev_feature(config.dev_data_path)

100%|██████████| 9011/9011 [00:04<00:00, 1864.21it/s]


In [0]:
test_dataset, test_df = dataProcessor.get_test_feature(config.test_data_path)

100%|██████████| 10000/10000 [00:05<00:00, 1896.58it/s]


In [0]:
model = RoBertModel_Base(config.bert_model_path, bert_config, config.max_seq_len).to(config.device)

Linear
Linear


In [11]:
model = Ernie_poollast3_multidp(config.bert_model_path, bert_config, config.max_seq_len).to(config.device)

Linear
Linear


In [0]:
for name, params in model.named_parameters():
    print(name, ":", params.size())

In [0]:
model = BertModel_Base(config.bert_model_path, bert_config, config.max_seq_len).to(config.device)

Linear
Linear


In [0]:
modelExcuter = ModelExcuter(train_dataset, dev_dataset, config)

In [0]:
modelExcuter.train(model, use_weight=False)

epoch [1/3]
Epoch: 1 Iter:    500, Train Loss:  0.64, Train Acc: 75.000%, Dev Loss:  0.72, Dev Acc: 68.649%, f1_score: 0.6233427, Time: 0:04:26 *
Epoch: 1 Iter:   1000, Train Loss:   0.5, Train Acc: 81.250%, Dev Loss:  0.63, Dev Acc: 72.123%, f1_score: 0.684644, Time: 0:09:52 *
Epoch: 1 Iter:   1500, Train Loss:  0.54, Train Acc: 87.500%, Dev Loss:  0.62, Dev Acc: 72.667%, f1_score: 0.6998514, Time: 0:15:10 *
Epoch: 1 Iter:   2000, Train Loss:  0.55, Train Acc: 81.250%, Dev Loss:  0.65, Dev Acc: 72.045%, f1_score: 0.6954341, Time: 0:20:29  
Epoch: 1 Iter:   2500, Train Loss:  0.38, Train Acc: 81.250%, Dev Loss:   0.6, Dev Acc: 73.699%, f1_score: 0.6990184, Time: 0:25:47  
Epoch: 1 Iter:   3000, Train Loss:  0.39, Train Acc: 87.500%, Dev Loss:   0.6, Dev Acc: 74.054%, f1_score: 0.7063076, Time: 0:31:05 *
Epoch: 1 Iter:   3500, Train Loss:  0.44, Train Acc: 81.250%, Dev Loss:   0.6, Dev Acc: 73.377%, f1_score: 0.7113082, Time: 0:36:24 *
Epoch: 1 Iter:   4000, Train Loss:  0.57, Train Acc

In [0]:
modelExcuter.predict(model, test_dataset, test_df['微博id'])

100%|██████████| 625/625 [01:29<00:00,  7.00it/s]


Time usage: 0:01:29
finish !


In [0]:
!nvidia-smi

Mon Apr 13 04:00:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!ps -aux

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  39192  6384 ?        Ss   09:54   0:00 /bin/bash -e 
root           8  0.0  0.4 691860 62368 ?        Sl   09:54   0:11 /tools/node/b
root          23  0.2  0.7 405724 101704 ?       Sl   09:54   0:29 /usr/bin/pyth
root         113  0.0  0.0  35884  4748 ?        Ss   09:55   0:00 tail -n +0 -F
root         122 81.2 37.2 34854896 4972520 ?    Ssl  09:55 169:14 /usr/bin/pyth
root         157  0.0  0.0  18376  1480 ?        S    09:55   0:00 /bin/bash --n
root         158  0.0  0.1 2174204 15780 ?       Sl   09:55   0:00 /opt/google/d
root         159  0.0  0.0  11464  1004 ?        S    09:55   0:00 grep --color=
root         162  0.4  0.6 2447308 83276 ?       Sl   09:55   0:54 /opt/google/d
root         214  0.0  0.0  18376  2988 ?        S    09:55   0:00 bash -c tail 
root         215  0.0  0.0   4568   804 ?        S    09:55   0:00 tail -n +0 -F
root         216  0.0  0.0  11464 

In [0]:
!kill -9 122

In [0]:
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);  

setInterval(()=>{
	if(Array.from(document.getElementById("connect").children[0].children[2].innerHTML).splice(3,4).toString() === '重,新,连,接'){
		document.getElementById("connect").children[0].children[2].click()
	}
},20000)